In [84]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esrao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
 data = pd.read_csv("C:/Users/esrao/makineogrenmesi/archive(4)/gender-classifier-DFE-791531.csv",
                encoding='latin1')

In [86]:
data.head(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [87]:
data.shape

(20050, 26)

In [88]:
data.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'gender', 'gender:confidence', 'profile_yn',
       'profile_yn:confidence', 'created', 'description', 'fav_number',
       'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'profileimage',
       'retweet_count', 'sidebar_color', 'text', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

In [90]:
# kullanacağımız datayı ayıralım
data = pd.concat([data.gender,data.description],axis = 1) 
data

,gender,description
0,male,i sing my own rhythm.
1,male,I'm the author of novels filled with family dr...
2,male,louis whining and squealing and all
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...
...,...,...
20045,female,(rp)
20046,male,"Whatever you like, it's not a problem at all. ..."
20047,male,#TeamBarcelona ..You look lost so you should f...
20048,female,Anti-statist; I homeschool my kids. Aspiring t...


In [91]:
# `brand` değerlerini çıkar, yalnızca `male` ve `female` olanları bırak
data = data[data['gender'].isin(['male', 'female'])]

In [92]:
print(data.isnull().sum())


gender            0
description    1700
dtype: int64


In [93]:
data.dropna(axis = 0, inplace = True)

C:\Users\esrao\AppData\Local\Temp\ipykernel_15948\2718241835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(axis = 0, inplace = True)


In [94]:
print(data.isnull().sum())


gender         0
description    0
dtype: int64


In [95]:
# 'gender' sütunundaki değerleri sayısal yapıyoruz.
data['gender'] = data['gender'].replace({'female': 1, 'male': 0})
print(data['gender'])

0        0
1        0
2        0
3        0
4        1
        ..
20045    1
20046    0
20047    0
20048    1
20049    1
Name: gender, Length: 11194, dtype: int64


C:\Users\esrao\AppData\Local\Temp\ipykernel_15948\4136736997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gender'] = data['gender'].replace({'female': 1, 'male': 0})


In [96]:
import re
def clean(review):
    # Tüm harf dışındaki karakterleri boşlukla değiştir
    review = re.sub('[^a-zA-Z ]', ' ', review)  # Boşluk karakterini koruyarak
    review = review.lower()  # Küçük harfe dönüştür
    return review

data['descrip_cleaned'] = pd.DataFrame(data['description'].apply(lambda x: clean(x)))

print(data.tail())

       gender                                        description  \
20045       1                                               (rp)   
20046       0  Whatever you like, it's not a problem at all. ...   
20047       0  #TeamBarcelona ..You look lost so you should f...   
20048       1  Anti-statist; I homeschool my kids. Aspiring t...   
20049       1                     Teamwork makes the dream work.   

                                         descrip_cleaned  
20045                                                rp   
20046  whatever you like  it s not a problem at all  ...  
20047   teambarcelona   you look lost so you should f...  
20048  anti statist  i homeschool my kids  aspiring t...  
20049                     teamwork makes the dream work   


C:\Users\esrao\AppData\Local\Temp\ipykernel_15948\4011610114.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['descrip_cleaned'] = pd.DataFrame(data['description'].apply(lambda x: clean(x)))


In [97]:

# Stop kelimelerini yükle
stop_words = set(stopwords.words('english'))

# Stop kelimelerini kaldırma fonksiyonu
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)

In [98]:
# Her sütuna stopword uygula
data['descrip_cleaned'] = data['descrip_cleaned'].apply(remove_stopwords)
data.head()

C:\Users\esrao\AppData\Local\Temp\ipykernel_15948\1003377735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['descrip_cleaned'] = data['descrip_cleaned'].apply(remove_stopwords)


,gender,description,descrip_cleaned
0,0,i sing my own rhythm.,sing rhythm
1,0,I'm the author of novels filled with family dr...,author novels filled family drama romance
2,0,louis whining and squealing and all,louis whining squealing
3,0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",mobile guy ers shazam google kleiner perkins y...
4,1,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,ricky wilson best frontman kaiser chiefs best ...


In [99]:
data = data.drop("description", axis=1)



In [100]:
data.head()

,gender,descrip_cleaned
0,0,sing rhythm
1,0,author novels filled family drama romance
2,0,louis whining squealing
3,0,mobile guy ers shazam google kleiner perkins y...
4,1,ricky wilson best frontman kaiser chiefs best ...


In [101]:
# 1. Metni özelliklere dönüştür, en sık geçen 500 kelimeyi seç
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(data['descrip_cleaned']).toarray()

In [102]:
# 2. Hedef değişkeni ayır ve int türüne dönüştür
y = data['gender']


In [103]:
# 3. Veriyi eğitim ve test setine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [104]:
# 4. GaussianNB modelini eğit
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [105]:
# 5. Modeli test et ve doğruluğu hesapla
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Model Accuracy:", accuracy)

Model Accuracy: 0.6114336757481018
